### Saving time-averages of the data in separate files

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
from xgcm import Grid
import filter
from dask.diagnostics import ProgressBar
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import glob, os

from dask.distributed import Client
from dask.distributed import LocalCluster
cluster = LocalCluster()
client = Client(cluster)

client

Client Scheduler: tcp://127.0.0.1:38018 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 16 Memory: 406.01 GB


#### Read Data first

In [2]:
path = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20201120/CM4_piControl_c192_OM4p125/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly/ts/monthly/5yr/"

ds_full = xr.open_mfdataset(path + "ocean_monthly.000101-000512*.nc")
ds_full = ds_full.isel(xq = slice(1,2881), yq=slice(1,2241))

ppdir1 = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20201120/CM4_piControl_c192_OM4p125/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly/"

ds_grid = xr.open_dataset(ppdir1 + "ocean_monthly.static.nc")
ds_grid = ds_grid.isel(xq = slice(0,2880), yq=slice(0,2240))

ds_full = xr.merge([ds_full, ds_grid])

In [3]:
path2 = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20201120/CM4_piControl_c192_OM4p125/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/ts/annual/10yr/"

ds_vmo = xr.open_dataset(path2 + "ocean_annual_z.0001-0010.vmo.nc")
ds_umo = xr.open_dataset(path2 + "ocean_annual_z.0001-0010.umo.nc")

In [4]:
print(ds_full)

<xarray.Dataset>
Dimensions:                 (nv: 2, time: 60, xh: 2880, xq: 2880, yh: 2240, yq: 2240, zi: 76)
Coordinates:
  * nv                      (nv) float64 1.0 2.0
  * time                    (time) object 0001-01-16 12:00:00 ... 0005-12-16 ...
  * xh                      (xh) float64 -298.6 -298.5 -298.4 ... 61.16 61.28
  * yh                      (yh) float64 -83.62 -83.58 -83.54 ... 89.92 89.97
  * xq                      (xq) float64 -298.5 -298.4 -298.3 ... 61.23 61.35
  * yq                      (yq) float64 -83.6 -83.56 -83.52 ... 89.95 90.0
  * zi                      (zi) float64 0.0 2.0 4.0 ... 3.722e+03 6.5e+03
Data variables:
    Heat_PmE                (time, yh, xh) float32 dask.array<chunksize=(60, 2240, 2880), meta=np.ndarray>
    average_DT              (time) timedelta64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T1              (time) object dask.array<chunksize=(60,), meta=np.ndarray>
    average_T2              (time) object dask.array<ch

In [18]:
drop_var = ['Heat_PmE', 'LwLatSens', 'col_mass', 'evs', 'ficeberg', 'friver','fsitherm','heat_content_cond', 
           'heat_content_fprec','heat_content_massin', 'heat_content_massout', 'heat_content_surfwater',
           'hfds', 'hfevapds', 'hfibthermds', 'hflso', 'hfrainds', 'hfrunoffds', 'hfsifrazil', 'hfsnthermds', 
           'hfsso', 'mass_wt', 'net_heat_coupler', 'net_massin', 'net_massout', 'nonpenSW', 'opottempmint',
          'prlq', 'prsn', 'rlntds', 'rsdo', 'rsntds', 'sfdsi', 'somint', 'tauuo', 'tauvo', 'ustar']

ds_save = ds_full.drop(drop_var)

ds_save = ds_save.mean('time')

distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing


In [21]:
vmo_2d = (ds_vmo['vmo'].isel(yq=slice(1,2241),time=slice(0,5)).mean('time')).sum('z_l')

umo_2d = (ds_umo['umo'].isel(xq=slice(1,2881),time=slice(0,5)).mean('time')).sum('z_l')

ds_save = xr.merge([ds_save, vmo_2d, umo_2d])

In [15]:
#print(list(ds_full.keys()))

In [24]:
print(ds_save)

ds_save.to_netcdf("/work/Hemant.Khatri/MOM6_idealised/MOM_budget/Global_125/Mom_Budget_mean_0001-0005.nc")

<xarray.Dataset>
Dimensions:           (nv: 2, xh: 2880, xq: 2880, yh: 2240, yq: 2240, zi: 76)
Coordinates:
  * nv                (nv) float64 1.0 2.0
  * xh                (xh) float64 -298.6 -298.5 -298.4 ... 61.05 61.16 61.28
  * yh                (yh) float64 -83.62 -83.58 -83.54 ... 89.86 89.92 89.97
  * xq                (xq) float64 -298.5 -298.4 -298.3 ... 61.12 61.23 61.35
  * yq                (yq) float64 -83.6 -83.56 -83.52 ... 89.89 89.95 90.0
  * zi                (zi) float64 0.0 2.0 4.0 ... 3.461e+03 3.722e+03 6.5e+03
Data variables:
    average_DT        timedelta64[ns] dask.array<chunksize=(), meta=np.ndarray>
    col_height        (yh, xh) float32 dask.array<chunksize=(2240, 2880), meta=np.ndarray>
    hf_CAu_2d         (yh, xq) float32 dask.array<chunksize=(2240, 2880), meta=np.ndarray>
    hf_CAv_2d         (yq, xh) float32 dask.array<chunksize=(2240, 2880), meta=np.ndarray>
    hf_PFu_2d         (yh, xq) float32 dask.array<chunksize=(2240, 2880), meta=np.ndarray>


In [23]:
print(ds_save.nbytes/1e9)

1.393541752


In [25]:
ds_save.close()
ds.umo.close()
ds.vmo.close()
ds_full.close()
client.close()
cluster.close()